In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*

In [52]:
import wave
import numpy as np
import struct
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
def get_wav_info(wav_file):
    """
    wavファイルの各情報を取得する関数．
    
    Parameters
    ----------
    wave_file : object
        wav形式のファイルを読み込んだもの
    
    returns
    ----------
    n_channels : int
        チャネル数:monoなら1，stereoなら2，5.1chなら6(たぶん)
    sample_width : int
        音声データ1サンプルあたりのバイト数
    framerate : int
        サンプリング周波数
    n_frames : int
        音声のデータ点の数
    """
    # チャネル数:monoなら1，stereoなら2，5.1chなら6(たぶん)
    n_channels = wav_file.getnchannels()

    # 音声データ1サンプルあたりのバイト数．2なら2bytes(16bit)，3なら24bitなど
    sample_width = wav_file.getsampwidth()

    # サンプリング周波数，普通のCDなら44.1k
    framerate = wav_file.getframerate()

    # 音声のデータ点の数
    n_frames = wav_file.getnframes()
    
    return n_channels, sample_width, framerate, n_frames

In [54]:
# バイナリ形式で開く
with open('./P_file/surround_5.1.raw', 'rb') as raw:
    # h:C言語でのshort型，<:リトルエンディアンで読み込む
    raw_data = np.fromfile(raw, dtype='<h', sep='')
print(raw_data[:13])

[ 2493  -603  2480  2480  2579  1402  3220 -1060  3129  3129  3138  1416
  3949]


In [55]:
# L(左前)，C(中央)，R(右前)，LS(左後)，RS(右後)，LFE(スーパーウーファ)
# 上記の全6ch分のデータが格納されているので，それぞれに分ける
left = raw_data[::6]
center = raw_data[1::6]
right = raw_data[2::6]
left_back = raw_data[3::6]
right_back = raw_data[4::6]
woofer = raw_data[5::6]

In [58]:
positions_dict = {'L': left, 'C': center, 'R': right,
             'LS': left_back, 'RS': right_back, 'LFE': woofer}

for position, data in positions_dict.items():
    # 今回はいらない，npとかで小数点の値で生成している場合に使う
    # left_wave = [int(x * 32767.0) for x in left]           # 16bit符号付き整数に変換
    binwave = struct.pack('h' * len(data), *data)  # バイナリ化

    with wave.Wave_write('./P_file/surround_5.1_' + position + '.wav') as wav:
        # チャンネル数(1:モノラル,2:ステレオ)，サンプルサイズ(バイト)，サンプリング周波数，フレーム数
        params = (1, 2, 16000, len(binwave), 'NONE', 'not compressed')
        wav.setparams(params)
        wav.writeframes(binwave)

In [59]:
# データのチェック
file_path = './P_file/surround_5.1_L.wav'

with wave.open(file_path, 'r') as wav:
    n_channels, sample_width, framerate, n_frames = get_wav_info(wav)
    buf = wav.readframes(1024)  # 一部を読み込む場合
#     buf = wav.readframes(-1)  # 全て読み込む場合

print(buf[:5])  # バイナリで返ってくる
print('channels :', n_channels)
print('sample size :', sample_width)
print('sampling rate :', framerate)
print('data points :', n_frames)

b'\xbd\t\x94\x0cm'
channels : 1
sample size : 2
sampling rate : 16000
data points : 80000
